# Bachelor Thesis

© 2026 Yvan Richard   
University of St. Gallen, Spring Term 2026

## Observations of Raw CRSP Data (2012-2019)

---

## 1. Presentation

In this notebook, I retrieve important information about the raw data I extracted from the CRSP daily file via WRDS. These observations will be used to design and code the data cleaning and processing pipelines.

## 2. Loading Raw Data

In [206]:
# libs
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

Once the libraries are loaded, I proceed to with loading the data in a pandas DataFrame.

In [207]:
# load data
df = pd.read_csv("../../data/raw/CRSP_2012_2019.csv")

/var/folders/7v/_v_y1jpx0rl056gg5rkjsw4r0000gn/T/ipykernel_2409/4204763773.py:2: DtypeWarning: Columns (6,9,10,15,19) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../../data/raw/CRSP_2012_2019.csv")


### 2.1. Basic Structure Info

In this section, I quickly look up some basic metadata.

In [208]:
# columns
print(df.columns)

Index(['PERMNO', 'date', 'NAMEENDT', 'SHRCD', 'EXCHCD', 'TICKER', 'SHRCLS',
       'PERMCO', 'DLSTCD', 'DLRETX', 'DLRET', 'BIDLO', 'ASKHI', 'PRC', 'VOL',
       'RET', 'SHROUT', 'CFACPR', 'CFACSHR', 'RETX'],
      dtype='object')


In [209]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14442404 entries, 0 to 14442403
Data columns (total 20 columns):
 #   Column    Dtype  
---  ------    -----  
 0   PERMNO    int64  
 1   date      object 
 2   NAMEENDT  object 
 3   SHRCD     float64
 4   EXCHCD    float64
 5   TICKER    object 
 6   SHRCLS    object 
 7   PERMCO    int64  
 8   DLSTCD    float64
 9   DLRETX    object 
 10  DLRET     object 
 11  BIDLO     float64
 12  ASKHI     float64
 13  PRC       float64
 14  VOL       float64
 15  RET       object 
 16  SHROUT    float64
 17  CFACPR    float64
 18  CFACSHR   float64
 19  RETX      object 
dtypes: float64(10), int64(2), object(8)
memory usage: 2.2+ GB


In [210]:
# length of data
print(f"Number of rows: {len(df)}")

Number of rows: 14442404


In [211]:
# number of rows with shrcd == 10 || 11
print(f"Number of rows with shrcd == 10 || 11: {len(df[(df['SHRCD'] == 10) | (df['SHRCD'] == 11)])}")

# keep only rows with shrcd == 10 || 11
df_ord = df[(df['SHRCD'] == 10) | (df['SHRCD'] == 11)].copy()

Number of rows with shrcd == 10 || 11: 7488775


This is quite important because for my thesis I exclusively focus on ordinary shares. 

In [212]:
# number of unique firms
print(f"Number of unique firms: {df_ord['PERMNO'].nunique()}")

# number of unique ticker symbols
print(f"Number of unique ticker symbols: {df_ord['TICKER'].nunique()}")

Number of unique firms: 5562
Number of unique ticker symbols: 5931


### 2.2. Exchange Codes

In [213]:
# exchange codes
print(f"Exchange codes: {df_ord['EXCHCD'].unique()}")

Exchange codes: [ 2.  3. -2.  1.  0. -1.  5.  6.]


-2	Halted by the NYSE or AMEX
-1	Suspended by the NYSE, AMEX, or NASDAQ
0	Not Trading on NYSE, AMEX, or NASDAQ
1	New York Stock Exchange
2	American Stock Exchange
3	The Nasdaq Stock Market(SM)
4	The Arca Stock Market(SM)
5	Mutual Funds (As Quoted by NASDAQ)

In [214]:
# paste the date into a datetime format
df_ord['date'] = pd.to_datetime(df_ord['date'], format='%Y-%m-%d')

Once the date are parsed, I select only the subsample of December 2012:

In [215]:
dec_2012 = df_ord[df_ord['date'] == '2012-12-17']

# print length of dec_2012
print(f"Number of rows in dec_2012: {len(dec_2012)}")

# number of unique stocks in dec_2012
print(f"Number of unique stocks in dec_2012: {dec_2012['PERMNO'].nunique()}")

Number of rows in dec_2012: 3757
Number of unique stocks in dec_2012: 3757


According to Bali et al. (2016):

> "In December 2012, the CRSP sample is comprised of 3675 stocks, of which 1319 are listed on the NYSE (1), 226 are listed on the AMEX (2), and 2130 are listed on the NASDAQ (3), and 2 are listed on other exchanges."

In [216]:
# if we keep stocks only listed on the NYSE, AMEX and NASDAQ, we have:
ex_codes = [1, 2, 3]
dec_2012_ex = dec_2012[dec_2012['EXCHCD'].isin(ex_codes)]

# number of unique stocks in dec_2012_ex
print(f"Number of unique stocks in dec_2012_ex: {dec_2012_ex['PERMNO'].nunique()}")

# value counts of exchange codes in dec_2012_ex
print(f"Value counts of exchange codes in dec_2012_ex:\n{dec_2012_ex['EXCHCD'].value_counts().sort_index()}")



Number of unique stocks in dec_2012_ex: 3672
Value counts of exchange codes in dec_2012_ex:
EXCHCD
1.0    1318
2.0     226
3.0    2128
Name: count, dtype: int64


This observation is well within the range of what has been described by Bali et al. (2016). This is therefore quite reassuring.

### 2.3. Stock Returns

In [217]:
# sort by date and PERMNO
df_ord.sort_values(by=['date', 'PERMNO'], inplace=True)

Once we have correctly sorted the data set, we must look for non-nan values of `DLRET` and `DLSTCD`. These values will guide how we handle the returns of delisted stocks. For my thesis, I will take the same approach as Bali et al. (2016, pp. 111-112):

> "To deal with delistings, CRSP maintains a monthly delistings (msedelist) file. The file includes, among other things, the date of the stock delisting (DLSTDT field), a code indicating the reason for the delisting (DLSTCD field), and the return realized by an investor who bought the stock at the last traded price in the previous month and held the stock through the delisting (DLRET field). The DLRET field is known as the delisting return. When possible, CRSP determines the price of the stock after the delisting, calculates the return of the stock based on this price (adjusted for any distributions), and reports thisvalue inthe DLRET field. Unfortunately, in many cases, CRSP is not able to determine a post-delisting value of the stock. In such cases, the DLRET field is missing. To handle these situations, we adjust returns for delisting using an approach suggested by Shumway (1997). Specifically, if a delisting return is available in CRSP, we take the return of the stock to be the delisting return. If a delisting return is unavailable, we rely on the reason for the delisting, as indicated by the DLSTCD field, to determine the return. If the DLSTCD is 500, 520, between 551 and 573 inclusive, 574, 580, or 584, we take the stock’s return during the delisting month to be−30%. If the delisting return is not available and DLSTCD has any value other than those mentioned in the previous sentence, we take the return of the stock in the delisting month to be −100%.".

In [218]:
# restrict the analysis to EXCHCD 1, 2, 3
ex_codes = [1, 2, 3]
df_ord_ex = df_ord[df_ord['EXCHCD'].isin(ex_codes)].copy()

Once we have correctly filtered for the ordinary shares and exchange code, we can look at the `RET` variable, which will be central in our thesis.

In [219]:
# check for NA values
print(f"Number of NA values in RET df_ord_ex:\n{df_ord_ex['RET'].isna().sum()}")

Number of NA values in RET df_ord_ex:
2647


We further check for letters:

In [220]:
print(f"Number of 'B', 'C' values in RET df_ord_ex:\n{df_ord_ex['RET'].isin(['B', 'C']).sum()}")

Number of 'B', 'C' values in RET df_ord_ex:
1767


We check the number of time the varibale `DLRET` and `DLCD` are not null (adjustments for delisting)

In [221]:
# non null values in DLRET or DLCD
count = df_ord_ex[['DLRET','DLSTCD']].notna().any(axis=1).sum()
print(f"Number of rows with non-null DLRET or DLSTCD: {count}")

Number of rows with non-null DLRET or DLSTCD: 1881


In [222]:
# length of df_ord_ex
print(f"Number of rows in df_ord_ex: {len(df_ord_ex)}")

Number of rows in df_ord_ex: 7375001


**Cleaning Pipeline for RET**

In [223]:
df_ord_ex[df_ord_ex['RET'].isna()].head()

,PERMNO,date,NAMEENDT,SHRCD,EXCHCD,TICKER,SHRCLS,PERMCO,DLSTCD,DLRETX,DLRET,BIDLO,ASKHI,PRC,VOL,RET,SHROUT,CFACPR,CFACSHR,RETX
384557,12063,2012-01-03,NaN,11.0,3.0,PARD,NaN,9902,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6752922,75550,2012-01-03,NaN,11.0,3.0,PVSA,NaN,9351,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6850082,76060,2012-01-03,NaN,11.0,3.0,STBC,NaN,10429,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8958326,85158,2012-01-03,NaN,11.0,3.0,INXI,NaN,15581,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9189660,85857,2012-01-03,NaN,11.0,3.0,BARI,NaN,16022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [224]:
# CLEANING STEPS:
# 1. identify where DLRET and DLSTCD are not null, and impute according to Bali et al. (2016).
# 2. where we have 'B' or 'C' in RET, we mark those rows as NaN, as they are not usable for our analysis.
# 3. drop all rows with NA values in RET, as we cannot use those for our analysis.

# 1. identify where DLRET and DLSTCD are not null, and impute according to Bali et al. (2016).
codes = [500, 520, 574, 580, 584] + list(range(551, 574))

# 1) apply DLSTCD proxy first (only on DLSTCD rows)
mask_stcd = df_ord_ex['DLSTCD'].notna()
df_ord_ex.loc[mask_stcd, 'RET'] = -1
df_ord_ex.loc[mask_stcd & df_ord_ex['DLSTCD'].isin(codes), 'RET'] = -0.3

# 2) then overwrite with DLRET where available (dominates)
mask_dlret = df_ord_ex['DLRET'].notna()
df_ord_ex.loc[mask_dlret, 'RET'] = df_ord_ex.loc[mask_dlret, 'DLRET']

# 2. where we have 'B' or 'C' in RET, we mark those rows as NaN, as they are not usable for our analysis.
df_ord_ex.loc[df_ord_ex['RET'].isin([chr(i) for i in range(ord('A'), ord('Z') + 1)]), 'RET'] = np.nan

# 3. drop all rows with NA values in RET, as we cannot use those for our analysis.
df_ord_ex.dropna(subset=['RET'], inplace=True)

# check for NA values in RET after cleaning
print(f"Number of NA values in RET after cleaning df_ord_ex:\n{df_ord_ex['RET'].isna().sum()}")

# length of df_ord_ex after cleaning
print(f"Number of rows in df_ord_ex after cleaning: {len(df_ord_ex)}")


Number of NA values in RET after cleaning df_ord_ex:
0
Number of rows in df_ord_ex after cleaning: 7370523


In [225]:
# number of unique stocks in df_ord_ex after cleaning
print(f"Number of unique stocks in df_ord_ex after cleaning: {df_ord_ex['PERMNO'].nunique()}")

Number of unique stocks in df_ord_ex after cleaning: 5531


In [226]:
# convert RET to numeric
df_ord_ex['RET'] = pd.to_numeric(df_ord_ex['RET'])

summary statistics

In [228]:
df_ord_ex['RET'].describe()

count    7.370523e+06
mean     5.856850e-04
std      3.847091e-02
min     -9.583330e-01
25%     -1.144200e-02
50%      0.000000e+00
75%      1.160100e-02
max      8.748427e+00
Name: RET, dtype: float64

In [241]:
# flag RET values that are above 1
df_ord_ex['RET_above_1'] = df_ord_ex['RET'] > 1
print(f"Number of rows with RET above 1: {df_ord_ex['RET_above_1'].sum()}")

# flag RET values that are below -1
df_ord_ex['RET_below_minus_1'] = df_ord_ex['RET'] < -1
print(f"Number of rows with RET below -1: {df_ord_ex['RET_below_minus_1'].sum()}")

Number of rows with RET above 1: 513
Number of rows with RET below -1: 0


We have a few extreme returns but no negative returns which would indicate an error.

### 2.4. Volume, Price and Shares Oustanding

In [242]:
# check for missing values in VOL, PRC and SHROUT after cleaning
print(f"Number of NA values in VOL after cleaning df_ord_ex:\n{df_ord_ex['VOL'].isna().sum()}")
print(f"Number of NA values in PRC after cleaning df_ord_ex:\n{df_ord_ex['PRC'].isna().sum()}")
print(f"Number of NA values in SHROUT after cleaning df_ord_ex:\n{df_ord_ex['SHROUT'].isna().sum()}")

Number of NA values in VOL after cleaning df_ord_ex:
28
Number of NA values in PRC after cleaning df_ord_ex:
0
Number of NA values in SHROUT after cleaning df_ord_ex:
0


As one can notice, some measure of the `VOL` are Nan:

In [244]:
df_ord_ex.loc[df_ord_ex['VOL'].isna()].head(28)

,PERMNO,date,NAMEENDT,SHRCD,EXCHCD,TICKER,SHRCLS,PERMCO,DLSTCD,DLRETX,...,PRC,VOL,RET,SHROUT,CFACPR,CFACSHR,RETX,RET_flag,RET_above_1,RET_below_minus_1
5228452,21785,2012-08-01,NaN,11.0,2.0,CAW,NaN,6522,NaN,NaN,...,4.75,NaN,0.014737,6087.0,1.0000,1.0000,0.0,False,False,False
5443887,29058,2012-08-01,NaN,11.0,2.0,DXR,NaN,6213,NaN,NaN,...,9.24,NaN,0.024390,4196.0,1.0000,1.0000,0.024390,False,False,False
5482859,30840,2012-08-01,NaN,11.0,2.0,CUO,NaN,20513,NaN,NaN,...,14.05,NaN,-0.038001,1635.0,1.0000,1.0000,-0.038001,False,False,False
5639939,38746,2012-08-01,NaN,11.0,1.0,AXR,NaN,20193,NaN,NaN,...,6.07,NaN,-0.008170,5996.0,1.0000,1.0000,-0.00817,False,False,False
5793499,46068,2012-08-01,NaN,11.0,2.0,WSO,B,21893,NaN,NaN,...,67.54,NaN,-0.010258,4709.0,1.0000,1.0000,-0.010258,False,False,False
5811706,46834,2012-08-01,NaN,11.0,2.0,JOB,NaN,20793,NaN,NaN,...,0.43,NaN,0.023810,21700.0,0.1000,0.1000,0.023810,False,False,False
5885072,49322,2012-08-01,NaN,11.0,1.0,ARL,NaN,20098,NaN,NaN,...,2.12,NaN,0.014354,11525.0,1.0000,1.0000,0.014354,False,False,False
5905767,50243,2012-08-01,NaN,11.0,2.0,SVT,NaN,21603,NaN,NaN,...,7.86,NaN,0.000000,2387.0,1.0000,1.0000,0.0,False,False,False
6016031,54084,2012-08-01,NaN,11.0,2.0,HWG,NaN,20869,NaN,NaN,...,9.85,NaN,0.000000,1525.0,1.0000,1.0000,0.0,False,False,False
6024676,54244,2012-08-01,NaN,11.0,2.0,BDL,NaN,20733,NaN,NaN,...,8.21,NaN,0.002442,1860.0,1.0000,1.0000,0.002442,False,False,False


It is odd to see that all the NAs of the variable `VOL` come from the same day (not necessarily from the same exchange).
Where the RET is 0 we can deduct that the volume was likely of 0 that day and otherwise we will drop the other observations.

In [246]:
df_ord_ex.columns

Index(['PERMNO', 'date', 'NAMEENDT', 'SHRCD', 'EXCHCD', 'TICKER', 'SHRCLS',
       'PERMCO', 'DLSTCD', 'DLRETX', 'DLRET', 'BIDLO', 'ASKHI', 'PRC', 'VOL',
       'RET', 'SHROUT', 'CFACPR', 'CFACSHR', 'RETX', 'RET_flag', 'RET_above_1',
       'RET_below_minus_1'],
      dtype='object')

The info we have gathered so far are sufficient for implementing a defensible data pipeline.